In [263]:
import plotly 
plotly.tools.set_credentials_file(username='ptyagi2', api_key='j1ztLn1qlEh65N1kcK0X')
import plotly.plotly as py
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
plt.rcParams["figure.figsize"] = (1,1)

In [264]:
#Loading the grants dataset
import os
import csv
os.getcwd()
df = {}
with open('C:\\Users\\trive\\Desktop\\Acads\\Data Visualization\\project-spring2017\\part2\Grants.csv', "r", encoding="utf8") as f:
    reader = csv.reader(f)
    header = next(reader)
    for k in header:
        df[k] = []
    for row in reader:
        for k, v in zip(header, row):
            df[k].append(v)
df.keys()

dict_keys(['Program', 'Organization Type', 'First Name', 'Address 2', 'Last Name', 'Institution', 'City', 'Fiscal Year', 'Program Type', 'Project Type', 'Total Amount Awarded and Disbursed', 'Address', 'County Code', 'Organizational Unit', 'Applicant Matching Amount', 'Census Block', 'Award Period From', 'Organizational Unit Location', 'Requested Matching Amount', 'State', 'Total Amount Originally Awarded', 'Project Title', 'MCD Code', 'Location', 'Title of Principal Investigator', 'Log Number', 'Description', 'Address 3', 'Award Period To', 'Census Tract', 'CBSA Code', 'Zip Code', 'Award Date', 'Place Code', 'MSAD Code', 'Requested Award Amount'])

This notebook shows the distribution of grants across various states.The fields for grants that we are interested in looking at, are 'Requested Award Amount'and'Total Amount Awarded and Disbursed'.

In [265]:
#Removing the dollar sign and converting the amount columns into float 
df["Amount awarded and disbursed"] = []
df["Amount requested"] = []
for value in df["Total Amount Awarded and Disbursed"]:
    new_data = value.strip('$')
    df["Amount awarded and disbursed"].append(float(new_data)) 
    
for value in df["Requested Award Amount"]:
    new_data1 = value.strip('$')
    df["Amount requested"].append(float(new_data1))
#df["Total Amount Awarded and Disbursed"],df["Amount awarded and disbursed"],df["Requested Award Amount"],df["Amount requested"]

In [266]:
class Dataset:
    def __init__(self, data):
        self.data = data.copy()
    
    def columns(self):
        return self.data.keys()
    
    def convert(self, column, dtype):
        self.data[column] = np.array(self.data[column], dtype=dtype)
                      

In [267]:
grants = Dataset(df)

value_types = {'Zip code': 'int',
               'Fiscal Year': 'int',
               'Amount awarded and disbursed': 'float',
               'Amount requested': 'float',
               }
for col in grants.columns():
    grants.convert(col, value_types.get(col, "str"))
#df

In [268]:
scl = [[0.0, 'rgb(242,240,247)'],[0.2, 'rgb(218,218,235)'],[0.4, 'rgb(188,189,220)'],\
            [0.6, 'rgb(158,154,200)'],[0.8, 'rgb(117,107,177)'],[1.0, 'rgb(84,39,143)']]

Why did you choose this color scale?

In [269]:
grants.data.keys()

dict_keys(['Location', 'Organization Type', 'Address 2', 'Institution', 'City', 'Fiscal Year', 'Program Type', 'Program', 'Total Amount Awarded and Disbursed', 'Organizational Unit Location', 'Address 3', 'Requested Matching Amount', 'Census Block', 'MCD Code', 'Amount awarded and disbursed', 'Census Tract', 'Amount requested', 'Zip Code', 'Log Number', 'Award Date', 'Place Code', 'CBSA Code', 'First Name', 'Last Name', 'Organizational Unit', 'Address', 'State', 'Applicant Matching Amount', 'Total Amount Originally Awarded', 'Award Period From', 'County Code', 'Project Title', 'Title of Principal Investigator', 'Project Type', 'Description', 'Award Period To', 'MSAD Code', 'Requested Award Amount'])

Now,the dictionary is converted to a Data Frame to group by the fields -"Amount awarded and disbursed", "Amount requested" at State and Fiscal Year level. It is then converted back to a dictionary for further plotting.

In [305]:
def f(Year,Program):
    #Converting grants data into a dataframe and applying groupby function
    grants_df= pd.DataFrame(grants.data)
    if Program =='ALL':
        grants_df_year = grants_df[(grants_df['Fiscal Year'] == int(Year))]
    else:
        grants_df_year = grants_df[(grants_df['Fiscal Year'] == int(Year)) & (grants_df['Program'] == Program)]
    grants_df_2 = grants_df_year.groupby(['State', 'Fiscal Year', 'Program'])[['Amount awarded and disbursed', 'Amount requested']].sum()
    
    #Converting grants_df_2 to a dictionary
    grants_new = {}
    grants_new.setdefault('State', [])
    grants_new.setdefault('Fiscal Year', [])
    grants_new.setdefault('Program', [])
    grants_new.setdefault('Amount awarded and disbursed', [])
    grants_new.setdefault('Amount requested', [])
    for x in grants_df_2.itertuples():
        grants_new['State'].append(x[0][0])
        grants_new['Fiscal Year'].append(x[0][1])
        grants_new['Program'].append(x[0][2])
        grants_new['Amount awarded and disbursed'].append(x[1])
        grants_new['Amount requested'].append(x[2])     
    
    #Creating Heat map using Plotly
    data = [ dict(
            type='choropleth',
            colorscale = scl,
            #hoverinfo='text',
            autocolorscale = False,
            locations = grants_new['State'],
            z = grants_new['Amount awarded and disbursed'],
            text = grants_new['Program'],
            locationmode = 'USA-states',
            marker = dict(
                line = dict (
                    color = 'rgb(255,255,255)',
                    width = 2
                ) ),
            colorbar = dict(
                #autotick = False,
                #tickprefix = '$',
                title = "Total amount of grants awarded in USD"),
        ) ]
    layout = dict(
        title = 'Total amount of grants awarded by State',
        geo = dict(
            scope='usa',
            projection=dict( type='albers usa' ),
            showlakes = True,
            lakecolor = 'rgb(255, 255, 255)'),
             )
    fig = dict(data=data, layout=layout)
    return py.iplot(fig, filename = 'heatmap_1')

In [306]:
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
grants_df= pd.DataFrame(grants.data)
year_list = sorted(np.array(grants_df['Fiscal Year'].unique()).tolist())
program_list = sorted(np.array(grants_df['Program'].unique()).tolist())
program_list.append('ALL')
interact(f, Year = year_list, Program = program_list);

PlotlyRequestError: Hey there! You've hit one of our API request limits. 

To get unlimited API calls(10,000/day), please upgrade to a paid plan. 

UPGRADE HERE: https://goo.gl/i7glmM 

Thanks for using Plotly! Happy Plotting!

I am trying to include the following also in the heat map:
1. Non-US states like Northern Mariana Islands, Puerto Rico, U.S. Virgin Islands, Guam, District of Columbia, Micronesia, Marshall Islands, Palau,American Samoa
2. Adding the feature of multi-selection, so that the user can select more than one Programs/Years.